# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows in the csvreader 
print("The total number of rows in the csvreader is", len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


The total number of rows in the csvreader is 8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Total number of rows after rows without artist name were removed is", sum(1 for line in f))

Total number of rows after rows without artist name were removed is 6821


# Part II. Apache Cassandra coding portion of my project. 

## Now we are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Connect to the cassandra instance on the local machine (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, we need a session
session = cluster.connect()

#### Creating Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute(" \
        CREATE KEYSPACE IF NOT EXISTS udacity_ks \
            WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}")
except Exception as e:
    print("Error: creating Keysapce")
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity_ks')
except Exception as e:
    print("Error: Setting Keyspace")
    print(e)



### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we model the database tables on the queries you want to run.

## My queries will be created to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4. This query will be using a WHERE at sessionId = 338 AND itemInSession = 4, \
# which will constitute the composite PARTITION KEY and PRIMARY KEY for this query because a music session can only be \
# at a particular item per time. 

query1 = "CREATE TABLE IF NOT EXISTS song_playlist_session "
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length float, \
            PRIMARY KEY ((sessionId, itemInSession)))" 
try:
    session.execute(query1)
except Exception as e:
    print("Error: Creating table")
    print(e)         

In [9]:
# set up the CSV file to extract data for query1. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert data into the table
        try:
            query1 = "INSERT INTO song_playlist_session (sessionId, itemInSession, artist, song, length) "
            query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
            session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        except Exception as e:
            print("Error: Inserting data")
            print(e)

####  SELECT to verify that the data have been inserted into the table

In [14]:
# validate the model for query1
query1 = "SELECT artist, song, length FROM song_playlist_session \
                WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query1)
except Exception as e: 
    print("Error: Validating query 1")
    print(e)
    
# data in tabular form
t1 = PrettyTable(["Artist", "Song", "Length"])
for row in rows:
    t1.add_row([row.artist, row.song, row.length])
print(t1)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


In [31]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and \
# user (first and last name) for userid = 10, sessionid = 182. This query will use the WHERE clause \
# at userId = 10 AND sessionId = 182. These two columns shall be the composite PARTITION KEY for \
# the query. Since the song column shall be sorted by itemInSession, itemInSession column shall \
# be the Clustering column for this query

query2 = "CREATE TABLE IF NOT EXISTS user_playlist_session "
query2 = query2 + "(userId int, sessionId int, song text, itemInSession int, artist text, firstName text, \
            lastName text, PRIMARY KEY ((userId, sessionId), itemInSession)) \
            WITH CLUSTERING ORDER BY (itemInSession ASC)" 
try:
    session.execute(query2)
except Exception as e:
    print("Error: Creating table")
    print(e)                       

In [32]:
# set up the CSV file to extract data for query2. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert data into the table
        try:
            query2 = "INSERT INTO user_playlist_session (userId, sessionId, song, itemInSession, artist, \
                    firstName, lastName) "
            query2 = query2 + " VALUES  (%s, %s, %s, %s, %s, %s, %s)"
            session.execute(query2, (int(line[10]), int(line[8]), line[9], int(line[3]), line[0], \
                                    line[1], line[4]))
        except Exception as e:
            print("Error: Inserting data")
            print(e)

In [33]:
# validate the model for query2
query2 = "SELECT artist, song, firstName, lastName FROM user_playlist_session WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query2)
except Exception as e: 
    print("Error: Validating query2")
    print(e)

# data in tabular form
t2 = PrettyTable(["Artist", "Song", "FirstName", "LastName"])
for row in rows:
    t2.add_row([row.artist, row.song, row.firstname, row.lastname])

print(t2)

+-------------------+------------------------------------------------------+-----------+----------+
|       Artist      |                         Song                         | FirstName | LastName |
+-------------------+------------------------------------------------------+-----------+----------+
| Sebastien Tellier |                      Kilometer                       |   Sylvie  |   Cruz   |
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie  |   Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie  |   Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie  |   Cruz   |
+-------------------+------------------------------------------------------+-----------+----------+


In [34]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the 
# song 'All Hands Against His Own'. This query will use a WHERE clause at song = 'All Hands Against His Own'
# The song will be the PARTITION KEY, firstName and lastName will be Clustering columns to ensure the PRIMARY KEY is unique

query3 = "CREATE TABLE IF NOT EXISTS song_users "
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))" 
try:
    session.execute(query3)
except Exception as e:
    print("Error: Creating table")
    print(e)                                     

In [35]:
# set up the CSV file to extract data for query3. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # Insert data into the table
        try:
            query3 = "INSERT INTO song_users (song, userId, firstName, lastName) VALUES (%s, %s, %s, %s)"
            session.execute(query3, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print("Error: Inserting data")
            print(e)

In [36]:
# validate the model for query3
query3 = "SELECT firstName, lastname FROM song_users WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query3)
except Exception as e: 
    print("Error: Validating query3")
    print(e)
    
# data in tabular form
t3 = PrettyTable(["FirstName", "LastName"])
for row in rows:
    t3.add_row([row.firstname, row.lastname])

print(t3)

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [37]:
try:
    session.execute("DROP TABLE song_playlist_session")
    session.execute("DROP TABLE user_playlist_session")
    session.execute("DROP TABLE song_users")
except Exception as e:
    print("Error: Dropping the tables")
    print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()